## Introduction
In this notebook, we aim to predict how a brand will rank on the interbrand ranking based on its past ranking, frequency, and word embedding.

The following files are needed to run the codes:

1. the word embedding model file: `L10T50G100A1ngV_iter1.p` (sent via google drive because it is too large for github)
2. the word to id json: `w2id_glove_corpora_minc_100.json` (on github)
3. the interbrand ranking by year json: `interbrand_brand2rankvalue.json` (on github)

In [10]:
# import packages
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np 
import scipy as sp
import scipy.spatial
import scipy.linalg
import json
import os
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE, MDS
from datetime import datetime
from time import time
import pickle
import sys
import pandas as pd

In [6]:
## this is the sample code for using the word embedding model
# model file path
# modify according to your setup
file_dir = '/Users/vincent/GoogleDriveSync/NTUCourse/BerkeleyCourse/Ming Hsu Lab/various_embedding/pierre2/stereotyping_word2vec_nodata/scraping/nytimes_2011/'
dw2v_filepath = file_dir + 'L10T50G100A1ngV_iter4.p' 
w2id_path = file_dir + 'w2id_glove_corpora_1996-2019_minc_100.json'

# define dynamic word vec class 
class DynamicWordVec():
    def __init__(self, dw2v_filepath, w2id_path):
        with open(dw2v_filepath, 'rb') as f:
            self.wordvec_matrix = pickle.load(f)
            self.num_periods = len(self.wordvec_matrix)

        with open(w2id_path) as f:
            self.w2id = json.load(f)
    
    def get_vec(self, w, yr):
        return self.wordvec_matrix[yr][self.w2id[w], :]

    def sim_by_vec(self, v1, v2, sim_type='cosim'):
        if sim_type == 'inner':
            return np.inner(v1, v2)
        elif sim_type == 'cosim':
            return 1 - sp.spatial.distance.cosine(v1, v2)
        else:
            raise Exception('sim_type should be either "inner" or "cosim"')
    
    def sim_by_word_year(self, w1, y1, w2, y2, sim_type='cosim'):
        v1 = self.wordvec_matrix[y1][self.w2id[w1], :]
        v2 = self.wordvec_matrix[y2][self.w2id[w2], :]
        return self.sim_by_vec(v1, v2, sim_type=sim_type)
    
    def is_in_vocab(self, w):
        return w in self.w2id
    
    def most_similar_words_in_year(self, w1, y1, topn, y2, include_self, sim_type='cosim'):
        temp_dict = {}
        for word in self.w2id:
            cont = True
            if not include_self:
                if word == w1:
                    cont = False
            if cont:
                temp_dict[word] = self.sim_by_word_year(w1, y1, word, y2)
        return sorted(temp_dict.items(), key=lambda x: x[1], reverse=True)[:topn]

In [8]:
dw2v = DynamicWordVec(dw2v_filepath, w2id_path) # load the trained dynamic word embedding model

start_t = 1996 # the first year of the corpora
target_word = 'obama' # target word of your choice
target_word_yr = 2011 # the year of the target word, again, feel free to change it

result_word_yrs = range(2015, 2020) 

most_sim_words = [dw2v.most_similar_words_in_year(target_word, target_word_yr-start_t, 10, yr-start_t, True) for yr in result_word_yrs]
for i, items in enumerate(most_sim_words):
    # this example gets the most similar word of twitter-2011 in each year
    string_to_print = ['{}({:.2f})'.format(w, sim) for w, sim in items]
    print(i+start_t, string_to_print)

1996 ['obama(0.96)', 'obama’s(0.91)', 'president’s(0.83)', 'bush(0.82)', 'clinton(0.82)', 'congress(0.81)', 'campaign(0.80)', 'administration(0.80)', 'republicans(0.79)', 'support(0.78)']
1997 ['obama(0.96)', 'obama’s(0.92)', 'clinton(0.85)', 'donald(0.83)', 'trump(0.83)', 'trump’s(0.83)', 'bush(0.83)', 'campaign(0.82)', 'hillary(0.82)', 'presidentelect(0.82)']
1998 ['obama(0.91)', 'trump’s(0.89)', 'trump(0.87)', 'president’s(0.85)', 'obama’s(0.84)', 'barack(0.84)', 'administration(0.83)', 'presidentelect(0.81)', 'donald(0.80)', 'president(0.80)']
1999 ['trump(0.88)', 'trump’s(0.88)', 'obama(0.86)', 'president’s(0.83)', 'president(0.82)', 'administration(0.79)', 'congress(0.78)', 'presidential(0.78)', 'barack(0.76)', 'republicans(0.76)']
2000 ['trump(0.87)', 'trump’s(0.86)', 'president(0.81)', 'pelosi(0.81)', 'obama(0.81)', 'biden(0.79)', 'democrats(0.79)', 'president’s(0.79)', 'presidential(0.77)', 'congress(0.77)']


In [22]:
## Create dataset

print('read files')
dw2v = DynamicWordVec(dw2v_filepath, w2id_path)

with open('./interbrand_brand2freq.json') as f:
    brand2year2freq = json.load(f)

with open('./interbrand_brand2rankvalue.json') as f:
    brand2year2rankvalue = json.load(f)

with open('./glove_corpora_totalwordcount.json') as f:
    year2count = json.load(f)

brands = sorted(list(brand2year2freq))
brands = [b for b in brands if dw2v.is_in_vocab(b)]

Ts = range(2001, 2019)
corpurs_start_t = 1996
rank_if_not_list = 101
n_feature = 67

dataset = []
header = ['brand_year', 'brand', 'year'] + ['wordvec_{}'.format(i) for i in range(1, 51)] + \
    ['freq_t-4', 'freq_t-3', 'freq_t-2', 'freq_t-1', 'freq_t'] + \
    ['ratiopermille_t-4', 'ratiopermille_t-3', 'ratiopermille_t-2', 'ratiopermille_t-1', 'ratiopermille_t'] + \
    ['rank_t', 'rank_t+1', 'isonlist_t', 'isonlist_t+1']
print(len(header))
assert(len(header)==n_feature)
for brand in brands:
    for t in Ts:
        #print(brand, t)
        wordvec = dw2v.get_vec(brand, t-corpurs_start_t).tolist() # dim-50
        freq_tm4_to_t = [brand2year2freq[brand][str(s)] for s in range(t-4, t+1)]
        totalwordcount_tm4_to_t = [year2count[str(s)] for s in range(t-4, t+1)]
        ratiopermille_tm4_to_t = [freq*1000 / count for freq, count in zip(freq_tm4_to_t, totalwordcount_tm4_to_t)]

        rank_t = brand2year2rankvalue[brand].get(str(t), (rank_if_not_list,))[0] # if not on the list, rank=101
        rank_tp1 = brand2year2rankvalue[brand].get(str(t+1), (rank_if_not_list,))[0]

        ison_t = rank_t < rank_if_not_list
        ison_tp1 = rank_tp1 < rank_if_not_list

        brandyear = '{}-{}'.format(brand, t)

        current_row = [brandyear, brand, t] + wordvec + freq_tm4_to_t + ratiopermille_tm4_to_t + [rank_t, rank_tp1, ison_t, ison_tp1]
        current_row = [str(c) for c in current_row]
        assert(len(current_row)==n_feature)
        dataset.append(current_row)


dataset = [','.join(row)+'\n' for row in dataset]
dataset = [','.join(header)+'\n'] + dataset
with open('interbrand_dataset.csv', 'w') as f:
    for row in dataset:
        f.write(row)
print('done')

read files
67
